# AOG Image-to-Video Example

This notebook demonstrates how to use AOG Image-to-Video API to convert static images to dynamic videos.

⚠️ **Important**: This service typically uses cloud API (such as Alibaba Cloud Tongyi Wanxiang), requires pre-configuration

## Table of Contents
1. [Environment Setup](#1-Environment Setup)
2. [Helper Functions](#2-Helper Functions)
3. [Basic Video Generation](#3-Basic Video Generation)
4. [Different Dynamic Effects](#4-Different Dynamic Effects)
5. [Batch Processing](#5-Batch Processing)

## 1. Environment Setup

In [ ]:
import requests
import json
import base64
from typing import Optional, Dict, List
from pathlib import Path
from IPython.display import Image, Video, display
import time

# AOG service configuration
AOG_BASE_URL = "http://localhost:16688"
AOG_API_SPEC_VERSION = "v0.2"
DEFAULT_MODEL = "wan2.2-i2v-plus"

print("✅ Configuration complete")
print(f"📍 AOG service address: {AOG_BASE_URL}")
print(f"🤖 Default model: {DEFAULT_MODEL}")
print("☁️  Cloud service: Alibaba Cloud Tongyi Wanxiang")

## 2. Helper Functions

In [ ]:
def encode_image_to_base64(image_path: str) -> Optional[str]:
    """
    将图像文件编码为 base64 字符串
    
    Args:
        image_path: 图像文件路径
    
    Returns:
        base64 编码字符串
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"❌ 编码图像失败: {e}")
        return None

def call_aog_image_to_video(
    image: str,
    image_type: str = "path",
    prompt: str = "",
    model: str = DEFAULT_MODEL
) -> Optional[Dict]:
    """
    Call AOG Image-to-Video API
    
    Args:
        image: 输入图像(路径/URL/base64)
        image_type: 图像类型 (path/url/base64)
        prompt: 描述期望视频动态效果
        model: Model name
    
    Returns:
        API response data
    """
    try:
        url = f"{AOG_BASE_URL}/aog/{AOG_API_SPEC_VERSION}/services/image-to-video"
        
        payload = {
            "model": model,
            "image": image,
            "image_type": image_type
        }
        
        if prompt:
            payload["prompt"] = prompt
        
        print(f"📤 Sending request...")
        print(f"🖼️  图像类型: {image_type}")
        if prompt:
            print(f"📝 Tip词: {prompt}")
        print("⏳ 视频生成可能需要较长时间,请耐心等待...")
        
        response = requests.post(url, json=payload, timeout=300)
        response.raise_for_status()
        
        result = response.json()
        
        if result.get("id"):
            print("✅ 视频Generation successful")
            video_url = result.get("data")
            if video_url:
                print(f"🎬 视频 URL: {video_url}")
        else:
            print(f"⚠️  生成失败: {result}")
        
        return result
        
    except requests.exceptions.ConnectionError:
        print("❌ 无法连接到 AOG 服务")
        return None
    except requests.exceptions.Timeout:
        print("❌ Request timeout(视频生成通常需要较长时间)")
        return None
    except Exception as e:
        print(f"❌ 错误: {e}")
        return None

def download_video(url: str, save_path: str):
    """
    下载视频到本地
    
    Args:
        url: 视频 URL
        save_path: 保存路径
    """
    try:
        print(f"📥 开始下载视频...")
        response = requests.get(url, timeout=60, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        
        with open(save_path, 'wb') as f:
            if total_size == 0:
                f.write(response.content)
            else:
                downloaded = 0
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        downloaded += len(chunk)
                        progress = (downloaded / total_size) * 100
                        print(f"\r📊 下载进度: {progress:.1f}%", end="")
        
        print(f"\n✅ 视频已保存到: {save_path}")
        return True
    except Exception as e:
        print(f"\n❌ 下载失败: {e}")
        return False

def display_video(video_path: str):
    """
    在 notebook 中显示视频
    
    Args:
        video_path: 视频文件路径
    """
    if not video_path:
        print("⚠️  没有视频可显示")
        return
    
    try:
        if Path(video_path).exists():
            print(f"\n🎬 播放视频: {video_path}")
            display(Video(video_path, width=512))
        else:
            print(f"⚠️  视频文件不存在: {video_path}")
    except Exception as e:
        print(f"❌ 显示失败: {e}")

print("✅ Helper Functions定义完成")

## 3. Basic Video Generation

使用本地图像生成视频

In [ ]:
# 准备输入图像
# 请替换为您自己图像路径
input_image_path = "path/to/your/image.jpg"

# 检查文件是否存在
if not Path(input_image_path).exists():
    print(f"⚠️  请先准备一张图像,并将路径设置为: {input_image_path}")
    print("💡 Tip: 建议使用主体明确、清晰图像")
else:
    print("✅ 找到输入图像")
    print(f"📁 路径: {input_image_path}")
    
    # 显示原始图像
    print("\n🖼️  原始图像:")
    display(Image(filename=input_image_path, width=512))

In [ ]:
# Basic Video Generation示例
prompt = "让猫在草地上奔跑"

response = call_aog_image_to_video(
    image=input_image_path,
    image_type="path",
    prompt=prompt
)

if response and response.get("id"):
    video_url = response.get("data")
    if video_url:
        # 下载视频
        output_dir = Path("output_videos")
        output_dir.mkdir(exist_ok=True)
        
        video_path = output_dir / "video_1.mp4"
        if download_video(video_url, str(video_path)):
            # 显示视频
            display_video(str(video_path))

## 4. Different Dynamic Effects

尝试多种动态效果

In [ ]:
# 自然场景 - 云朵移动
prompt = "让云朵缓缓移动"

response = call_aog_image_to_video(
    image=input_image_path,
    image_type="path",
    prompt=prompt
)

if response and response.get("id"):
    video_url = response.get("data")
    if video_url:
        video_path = output_dir / "video_clouds.mp4"
        if download_video(video_url, str(video_path)):
            display_video(str(video_path))

In [ ]:
# 植物动态 - 树叶摇摆
prompt = "让树叶随风摇摆"

response = call_aog_image_to_video(
    image=input_image_path,
    image_type="path",
    prompt=prompt
)

if response and response.get("id"):
    video_url = response.get("data")
    if video_url:
        video_path = output_dir / "video_leaves.mp4"
        if download_video(video_url, str(video_path)):
            display_video(str(video_path))

In [ ]:
# 水面效果 - 涟漪
prompt = "让水面泛起涟漪"

response = call_aog_image_to_video(
    image=input_image_path,
    image_type="path",
    prompt=prompt
)

if response and response.get("id"):
    video_url = response.get("data")
    if video_url:
        video_path = output_dir / "video_water.mp4"
        if download_video(video_url, str(video_path)):
            display_video(str(video_path))

## 5. Batch Processing

为同一图像生成多种动态效果

In [ ]:
# 定义多种动态效果
effects = [
    "让猫在草地上奔跑",
    "让云朵缓缓移动",
    "让树叶随风摇摆",
    "让水面泛起涟漪"
]

all_videos = []

for i, effect in enumerate(effects, 1):
    print(f"\n🎬 生成效果 {i}/{len(effects)}: {effect}")
    print("=" * 50)
    
    response = call_aog_image_to_video(
        image=input_image_path,
        image_type="path",
        prompt=effect
    )
    
    if response and response.get("id"):
        video_url = response.get("data")
        if video_url:
            video_path = output_dir / f"video_{i}.mp4"
            if download_video(video_url, str(video_path)):
                all_videos.append({
                    "effect": effect,
                    "path": str(video_path)
                })
    
    # 避免请求过快
    if i < len(effects):
        print("\n⏳ 等待 5 秒后继续...")
        time.sleep(5)

print(f"\n✅ 成功生成 {len(all_videos)}/{len(effects)} 个视频")

In [ ]:
# 播放所有生成视频
for video_info in all_videos:
    print(f"\n🎬 效果: {video_info['effect']}")
    display_video(video_info['path'])

## 6. 使用 URL 输入

直接使用网络图像 URL

In [ ]:
# 使用网络图像 URL
image_url = "https://example.com/your-image.jpg"
prompt = "让猫在草地上奔跑"

response = call_aog_image_to_video(
    image=image_url,
    image_type="url",
    prompt=prompt
)

if response and response.get("id"):
    video_url = response.get("data")
    if video_url:
        video_path = output_dir / "video_from_url.mp4"
        if download_video(video_url, str(video_path)):
            display_video(str(video_path))

## 7. 无Tip词生成

让模型自动决定动态效果

In [ ]:
# 不提供Tip词,让模型自动生成
response = call_aog_image_to_video(
    image=input_image_path,
    image_type="path"
)

if response and response.get("id"):
    video_url = response.get("data")
    if video_url:
        video_path = output_dir / "video_auto.mp4"
        if download_video(video_url, str(video_path)):
            display_video(str(video_path))

## 🎉 Summary

Through this notebook, you have learned:

1. ✅ 调用 Image-to-Video API
2. ✅ 使用不同图像输入方式(路径/URL/Base64)
3. ✅ 应用多种动态效果
4. ✅ Batch Generation视频
5. ✅ 下载和播放生成视频

### Tip词建议

**动物动作**:
- 让猫在草地上奔跑
- 让鸟儿展翅飞翔
- 让狗摇尾巴

**自然场景**:
- 让云朵缓缓移动
- 让树叶随风摇摆
- 让水面泛起涟漪

**人物动作**:
- 让人物微笑并挥手
- 让人物转头看向镜头
- 让人物走向前方

**物体运动**:
- 让汽车缓缓驶过
- 让花朵绽放
- 让灯光闪烁

### 使用技巧

1. **图像选择**: 使用清晰、主体明确图像
2. **Tip词**: 描述具体动作或效果
3. **耐心等待**: 视频生成通常需要较长时间
4. **Batch Processing**: Note请求间隔,避免过快

### Next Steps

- 尝试更多创意Tip词
- 探索 [Image-to-Image](../image-to-image/) 服务
- Check out [Text-to-Image](../text-to-image/) 服务
- Read [AOG API 文档](../../docs/)